In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler  #standartlaştırma için eklendi
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors

In [2]:
#uyarı mesajlarıylaa karşılaşmamak için eklediğimiz bir fonksiyon
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
df=pd.read_csv(r"C:\Users\Onur\Desktop\Turkcell_Machinelearning\Hitters.csv")
df=df.dropna()
dms=pd.get_dummies(df[['League','Division','NewLeague']])

y=df["Salary"]
x_=df.drop(['Salary','League','Division','NewLeague'], axis=1).astype('float64')

x=pd.concat([x_,dms[['League_N','Division_W','NewLeague_N']]],axis=1)
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.25,random_state=42)

In [4]:
x_train.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
183,328.0,91.0,12.0,51.0,43.0,33.0,2.0,342.0,94.0,12.0,51.0,44.0,33.0,145.0,59.0,8.0,1,0,1
229,514.0,144.0,0.0,67.0,54.0,79.0,9.0,4739.0,1169.0,13.0,583.0,374.0,528.0,229.0,453.0,15.0,1,0,1
286,593.0,152.0,23.0,69.0,75.0,53.0,6.0,2765.0,686.0,133.0,369.0,384.0,321.0,315.0,10.0,6.0,0,1,0
102,233.0,49.0,2.0,41.0,23.0,18.0,8.0,1350.0,336.0,7.0,166.0,122.0,106.0,102.0,132.0,10.0,0,0,0
153,341.0,95.0,6.0,48.0,42.0,20.0,10.0,2964.0,808.0,81.0,379.0,428.0,221.0,158.0,4.0,5.0,1,1,1


Model & Tahmin

In [5]:
knn_model=KNeighborsRegressor().fit(x_train,y_train)

In [6]:
knn_model

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [7]:
knn_model.n_neighbors  #komşu sayısı

5

In [8]:
knn_model.metric  #metrice ulaşıyoruz

'minkowski'

In [9]:
dir(knn_model) #model nesnesinden alabileceğimiz şeyleri görüyoruz

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_algorithm_metric',
 '_estimator_type',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_get_tags',
 '_kneighbors_reduce_func',
 '_more_tags',
 '_pairwise',
 '_tree',
 '_y',
 'algorithm',
 'effective_metric_',
 'effective_metric_params_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params',
 'n_jobs',
 'n_neighbors',
 'n_samples_fit_',
 'p',
 'predict',
 'radius',
 'score',
 'set_params',
 'weights']

In [10]:
knn_model.predict(x_test)[0:5]

array([ 510.3334,  808.3334,  772.5   ,  125.5   , 1005.    ])

In [11]:
y_pred=knn_model.predict(x_test)

In [12]:
np.sqrt(mean_squared_error(y_test,y_pred))  #knn modeli ile elde ettiğimiz ilkel hata

426.6570764525201

MODEL TUNİNG

In [13]:
knn_model

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [14]:
RMSE=[]
for k in range(10):
    k=k+1
    knn_model=KNeighborsRegressor(n_neighbors=k).fit(x_train,y_train)
    y_pred=knn_model.predict(x_test)
    rmse=np.sqrt(mean_squared_error(y_test,y_pred))
    RMSE.append(rmse)
    print("k=",k,'için RMSE değeri =',rmse)

k= 1 için RMSE değeri = 455.03925390751965
k= 2 için RMSE değeri = 415.99629571490965
k= 3 için RMSE değeri = 420.6765370082348
k= 4 için RMSE değeri = 428.8564674588792
k= 5 için RMSE değeri = 426.6570764525201
k= 6 için RMSE değeri = 423.5071669008732
k= 7 için RMSE değeri = 414.9361222421057
k= 8 için RMSE değeri = 413.7094731463598
k= 9 için RMSE değeri = 417.84419990871265
k= 10 için RMSE değeri = 421.6252180741266


In [15]:
#GridSearchCV ile komşu sayısı bulma
#bu mmetod m.ö algoritmalarında belirlemek istediğmiz hiperparametrelerinin değerlerini belirlemek için kullanılan fonksiyondur

In [16]:
knn_params={'n_neighbors':np.arange(1,30,1)}

In [17]:
knn=KNeighborsRegressor()

In [18]:
knn_cv_model=GridSearchCV(knn,knn_params,cv=10).fit(x_train,y_train) #ilk argüman model,denenecek parametreleri yazıyoruz


In [19]:
knn_cv_model.best_params_   #en iyi parametreyi öğreniyoruz

{'n_neighbors': 8}

In [20]:
#Final Modeli

In [21]:
knn_tuned=KNeighborsRegressor(n_neighbors=knn_cv_model.best_params_["n_neighbors"]).fit(x_train,y_train)

In [22]:
y_pred=knn_tuned.predict(x_test)
np.sqrt(mean_squared_error(y_test,y_pred))

413.7094731463598